In [1]:
import os
import pandas as pd
from glob import glob
from sklearn.model_selection import StratifiedKFold

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
dataset_root = "Skin cancer ISIC The International Skin Imaging Collaboration/Train"

image_paths = []
labels = []

class_names = sorted(os.listdir(dataset_root))  # keep consistent class order

for label in class_names:
    class_dir = os.path.join(dataset_root, label)
    if os.path.isdir(class_dir):
        img_files = glob(os.path.join(class_dir, "*.jpg")) + \
                    glob(os.path.join(class_dir, "*.png")) + \
                    glob(os.path.join(class_dir, "*.jpeg"))
        image_paths.extend(img_files)
        labels.extend([label] * len(img_files))

df = pd.DataFrame({"image": image_paths, "label": labels})
print(df.head())
print("Total images:", len(df))
print("Classes:", class_names)


                                               image              label
0  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
1  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
2  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
3  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
4  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
Total images: 2239
Classes: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label_idx"] = le.fit_transform(df["label"])

num_classes = len(le.classes_)
print("Class to Index Mapping:", dict(zip(le.classes_, range(num_classes))))


Class to Index Mapping: {'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'seborrheic keratosis': 6, 'squamous cell carcinoma': 7, 'vascular lesion': 8}


In [4]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df["fold"] = -1
for fold, (_, val_idx) in enumerate(skf.split(df["image"], df["label_idx"])):
    df.loc[val_idx, "fold"] = fold

df["fold"].value_counts()


fold
3    448
0    448
1    448
2    448
4    447
Name: count, dtype: int64

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [6]:
class SkinCancerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, "image"]
        label = self.df.loc[idx, "label_idx"]

        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        return img, label


In [7]:
def get_dataloaders(fold, batch_size=32):
    train_df = df[df.fold != fold]
    val_df = df[df.fold == fold]

    train_dataset = SkinCancerDataset(train_df, transform=train_transform)
    val_dataset = SkinCancerDataset(val_df, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                            shuffle=False, num_workers=2)
    return train_loader, val_loader


In [8]:
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

def create_model(num_classes):
    weights = EfficientNet_B0_Weights.DEFAULT
    model = efficientnet_b0(weights=weights)

    # Freeze only early layers (blocks 0–4)
    for idx, block in enumerate(model.features):
        if idx < 5:
            for param in block.parameters():
                param.requires_grad = False
        else:
            for param in block.parameters():
                param.requires_grad = True

    # Replace classifier for 9 classes
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)

    return model.to(device)


In [9]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['label_idx']),
    y=df['label_idx']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion


CrossEntropyLoss()

In [10]:
def get_optimizer(model, lr=1e-3):
    # Only include parameters that require gradients
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(trainable_params, lr=lr, weight_decay=1e-5)
    return optimizer


In [11]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0
    preds, targets = [], []
    
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds.extend(outputs.argmax(dim=1).cpu().numpy())
        targets.extend(labels.cpu().numpy())

    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='macro')
    return total_loss/len(loader), acc, f1


def validate_one_epoch(model, loader):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Validating"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds.extend(outputs.argmax(dim=1).cpu().numpy())
            targets.extend(labels.cpu().numpy())

    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='macro')
    return total_loss/len(loader), acc, f1


In [12]:
EPOCHS = 10  # can increase later if time allows
BATCH_SIZE = 32

fold_performance = []

for fold in range(5):
    print(f"\n===== FOLD {fold+1} / 5 =====")
    
    train_loader, val_loader = get_dataloaders(fold, batch_size=BATCH_SIZE)
    model = create_model(num_classes)
    optimizer = get_optimizer(model)

    best_f1 = 0
    best_model_path = f"best_model_fold{fold}.pt"

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer)
        val_loss, val_acc, val_f1 = validate_one_epoch(model, val_loader)

        print(f"Train Loss: {train_loss:.4f} | ACC: {train_acc:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | ACC: {val_acc:.4f} | F1: {val_f1:.4f}")

        # Save best fold model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)
            print(">>> Saved best model so far!")

    fold_performance.append({
        "fold": fold,
        "best_f1": best_f1
    })

print("\n=== Final Fold Results ===")
print(pd.DataFrame(fold_performance))



===== FOLD 1 / 5 =====

Epoch 1/10


Validating: 100%|███████████████████████████████| 14/14 [00:03<00:00,  3.63it/s]


Train Loss: 1.6954 | ACC: 0.3439 | F1: 0.3257
Val   Loss: 1.4507 | ACC: 0.4308 | F1: 0.4442
>>> Saved best model so far!

Epoch 2/10


Validating: 100%|███████████████████████████████| 14/14 [00:03<00:00,  3.67it/s]


Train Loss: 1.3362 | ACC: 0.4813 | F1: 0.4575
Val   Loss: 1.3116 | ACC: 0.5268 | F1: 0.4611
>>> Saved best model so far!

Epoch 3/10


...

Train Loss: 0.9153 | ACC: 0.6004 | F1: 0.5900
Val   Loss: 1.1541 | ACC: 0.5615 | F1: 0.5361

Epoch 9/10


Validating: 100%|███████████████████████████████| 14/14 [00:03<00:00,  4.31it/s]


Train Loss: 0.8137 | ACC: 0.6484 | F1: 0.6372
Val   Loss: 1.0956 | ACC: 0.5906 | F1: 0.5672

Epoch 10/10


Validating: 100%|███████████████████████████████| 14/14 [00:03<00:00,  4.22it/s]

Train Loss: 0.7853 | ACC: 0.6635 | F1: 0.6483
Val   Loss: 1.1133 | ACC: 0.5727 | F1: 0.5384

=== Final Fold Results ===
   fold   best_f1
0     0  0.615159
1     1  0.596663
2     2  0.572322
3     3  0.617536
4     4  0.591186


In [15]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

# Paths
BEST_MODEL_PATH = "best_model_fold3.pt"  # <- Update fold if needed
TEST_DIR = "Skin cancer ISIC The International Skin Imaging Collaboration/Test"

# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Same transforms used in validation
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load test dataset
test_dataset = datasets.ImageFolder(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load model (EfficientNet B0)
from torchvision.models import efficientnet_b0
model = efficientnet_b0(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(test_dataset.classes))
model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
model.to(device)
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Metrics
print("\n=== TEST SET RESULTS ===")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes, digits=4))

cm = confusion_matrix(all_labels, all_preds)
cm_df = pd.DataFrame(cm, index=test_dataset.classes, columns=test_dataset.classes)

print("\n=== CONFUSION MATRIX ===")
print(cm_df)


/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_13690/504968727.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for 


=== TEST SET RESULTS ===
                            precision    recall  f1-score   support

         actinic keratosis     0.7500    0.3750    0.5000        16
      basal cell carcinoma     0.6111    0.6875    0.6471        16
            dermatofibroma     0.7059    0.7500    0.7273        16
                  melanoma     0.0000    0.0000    0.0000        16
                     nevus     0.4706    1.0000    0.6400        16
pigmented benign keratosis     0.4167    0.6250    0.5000        16
      seborrheic keratosis     0.0000    0.0000    0.0000         3
   squamous cell carcinoma     0.6667    0.3750    0.4800        16
           vascular lesion     0.7500    1.0000    0.8571         3

                  accuracy                         0.5424       118
                 macro avg     0.4857    0.5347    0.4835       118
              weighted avg     0.5100    0.5424    0.4956       118


=== CONFUSION MATRIX ===
                            actinic keratosis  basal cell car

/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag